In [1]:
import os

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
allfiles = []
for root, dirs, files in os.walk("/home/ubuntu/AutoLDA/Code/Transcripts/", topdown=False):
    for name in files:
        if name.endswith(".txt") and "checkpoint" not in name:
            allfiles.append(os.path.join(root, name))

In [4]:
len(allfiles)

555

In [5]:
def getAllText(allfiles):
    res = []
    for file in allfiles:
        with open(file,"r") as f:
            res.append(f.read())
    return " ".join(res)
            

In [6]:
allText = getAllText(allfiles)

In [7]:
len(allText)

5081360

In [8]:
allText[:20]

'  hey you guys welco'

In [9]:
processedText = allText.split()

In [10]:
processedText[:10]

['hey', 'you', 'guys', 'welcome', 'back', 'to', 'my', 'channel', 'is', 'your']

In [11]:
# Ref: https://github.com/facebookresearch/InferSent

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

In [14]:
# Load model
from InferSent.models import InferSent
model_version = 1
MODEL_PATH = "InferSent/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [15]:
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model

In [16]:
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'GloveFineTune/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [17]:
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [18]:
# # Load some sentences
# sentences = []
# with open('samples.txt') as f:
#     for line in f:
#         sentences.append(line.strip())
# print(len(sentences))

In [19]:
sentences =  allText.split(".")

In [20]:
len(sentences)

1893

In [21]:
model.encode(['the cat eats.'])

array([[ 0.13108404,  0.0095878 ,  0.00454948, ...,  0.02293484,
        -0.03814263, -0.01252607]], dtype=float32)

In [22]:
cutS = [s[:max(len(s),20)] for s in sentences]

In [23]:
embeddings = model.encode(cutS[:20], bsize=128, tokenize=False, verbose=True)
print('nb sentences encoded : {0}'.format(len(embeddings)))

Nb words kept : 24339/25879 (94.0%)


/home/ubuntu/AutoLDA/Code/Embeddings/InferSent/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)[idx_sort]


Speed : 0.7 sentences/s (cpu mode, bsize=128)
nb sentences encoded : 20


In [27]:
np.shape(embeddings)

(20, 4096)

In [24]:
np.linalg.norm(model.encode(['the cat eats.']))

3.5349097

In [ ]:
# Ref: https://github.com/facebookresearch/InferSent/blob/main/demo.ipynb

In [25]:
from models import InferSent
V = 1
MODEL_PATH = 'InferSent/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

ModuleNotFoundError: No module named 'models'